In [1]:
import numpy as np
import json

from ahyper import utils, annotated_hypergraph

In [2]:
with open('data/enron_hypergraph_annotated.json') as file:
    data = json.load(file)

roles = ['cc', 'from', 'to']

# add unique ID to each edge:
for i in range(len(data)):
    data[i]['eid'] = -(i+1)

In [3]:
data[0:2]

[{'cc': [],
  'date': '1998-11-13 12:07:00',
  'eid': -1,
  'from': [67],
  'to': [108]},
 {'cc': [],
  'date': '1998-11-19 15:19:00',
  'eid': -2,
  'from': [67],
  'to': [73]}]

# Construct an Annotated Hypergraph

In [4]:
A = annotated_hypergraph.annotated_hypergraph(data, roles)

First, `A` stores lists of the node and edge ids. Nodes are numbered from $0$ to $n-1$. Edges are numbered from $-1$ to $-m$. 

In [5]:
A.get_node_list()[0:10]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10])

In [6]:
A.get_edge_list()[0:10]

array([-10504, -10503, -10502, -10501, -10500, -10499, -10498, -10497,
       -10496, -10495])

We can also get the node degree sequence, optionally broken down by role: 

In [7]:
# degree sequence
A.node_degrees() # get all node degrees (totals)
A.node_degrees(by_role = True)[4] # get the role-degrees of node 4 

{'cc': 0, 'from': 12, 'to': 5}

Similarly, we can get the edge dimension sequence, again optionally broken down by role: 

In [8]:
# edge dimension sequence

A.edge_dimensions() # get all edge dimensions (totals)
A.edge_dimensions(by_role = True)[-5] # get the role-dimensions of edge -5

{'cc': 2, 'from': 1, 'to': 0}

Internally, `A` is representing the data as an annotated node-edge incidence list. It's convenient to think of this as the edge-list of the bipartite graph in which each edge is labeled with a name and a role. It is possible to access the list directly: 

In [9]:
A.get_IL()[0:10]

[[67, 'from', -1, '1998-11-13 12:07:00'],
 [108, 'to', -1, '1998-11-13 12:07:00'],
 [67, 'from', -2, '1998-11-19 15:19:00'],
 [73, 'to', -2, '1998-11-19 15:19:00'],
 [73, 'cc', -3, '1998-11-19 16:24:00'],
 [67, 'from', -3, '1998-11-19 16:24:00'],
 [108, 'cc', -4, '1998-11-24 10:23:00'],
 [96, 'cc', -4, '1998-11-24 10:23:00'],
 [22, 'cc', -4, '1998-11-24 10:23:00'],
 [67, 'from', -4, '1998-11-24 10:23:00']]

# Stub-Labeled MCMC 

We can define a simple version of stub-labeled Markov Chain Monte Carlo in this space, which essentially amounts to swapping edges of the bipartite graph in such a way that edges can only be swapped if their roles agree. This MCMC algorithm preserves degree sequence and edge dimension sequence, including the `by_role` variants. 

# Check for preservation of node degrees and edge dimensions

In [10]:
d0 = A.node_degrees(by_role = True)
k0 = A.edge_dimensions(by_role = True)

In [11]:
A.stub_labeled_MCMC(n_steps = 100000)
A.get_IL()[0:10] # not the same list as above

[[96, 'from', -1, '1998-11-13 12:07:00'],
 [112, 'to', -1, '1998-11-13 12:07:00'],
 [114, 'from', -2, '1998-11-19 15:19:00'],
 [62, 'to', -2, '1998-11-19 15:19:00'],
 [67, 'cc', -3, '1998-11-19 16:24:00'],
 [8, 'from', -3, '1998-11-19 16:24:00'],
 [45, 'cc', -4, '1998-11-24 10:23:00'],
 [114, 'cc', -4, '1998-11-24 10:23:00'],
 [39, 'cc', -4, '1998-11-24 10:23:00'],
 [40, 'from', -4, '1998-11-24 10:23:00']]

In [12]:
d = A.node_degrees(by_role = True)
k = A.edge_dimensions(by_role = True)

In [13]:
d0 == d, k0 == k # but the degree and dimension sequences are preserved

(True, True)

# Output

You can read out data from `A` either as a list of dicts ("records", suitable for output as json) or as an incidence list. 

In [14]:
A.get_records()[0:2]

[{'cc': [],
  'date': '1998-11-13 12:07:00',
  'eid': -1,
  'from': [96],
  'to': [112]},
 {'cc': [],
  'date': '1998-11-19 15:19:00',
  'eid': -2,
  'from': [114],
  'to': [62]}]

In [15]:
A.get_IL()[0:4]

[[96, 'from', -1, '1998-11-13 12:07:00'],
 [112, 'to', -1, '1998-11-13 12:07:00'],
 [114, 'from', -2, '1998-11-19 15:19:00'],
 [62, 'to', -2, '1998-11-19 15:19:00']]

# Observables

For an annotated hypergraph null model to make sense we need observables on the hypergraph that take the node roles into account. 

Check: clustering coefficients will also change upon a shuffle, however it will differ from the non-annotated case.
The clustering coefficients are only calculated on the simplified graph projection.

One observable is the local role density around a node.
For a focal node $i$, this is the sum of roles, over all neighbours of $i$, and over all edges. This can include the focal node, or not.
We expect that this varies strongly across a graph. 
For example, in a an authorship graph, an author who is first author on a large number of papers will more likely be surrounded by authors who are middle or last authors.

In [16]:
from ahyper.observables import local_role_density

In [17]:
A = annotated_hypergraph.annotated_hypergraph(data, roles)

densities = local_role_density(A, include_focus=False)
densities.head()

,cc,from,to
0,0.094737,0.294737,0.610526
1,0.102941,0.235294,0.661765
2,0.354331,0.145669,0.500000
3,0.234742,0.530516,0.234742
4,0.086957,0.217391,0.695652


If we want to consider a single value, we can calculate the normalised entropy of the role density. 
Here a value of zero indicates only 1 role is present in the neighbourhood, and a value of one indicates all roles are equally prevalent.

In [18]:
def entropy(arr):
    return -sum([x*np.log2(x)/np.log2(len(arr)) for x in arr if x>0])

densities.apply(entropy, axis=1).mean()

0.8012102776183179

In [19]:
# Perform a stub shuffle
A.stub_labeled_MCMC(n_steps = 100000)

In [20]:
densities = local_role_density(A, include_focus=False)
densities.head()

,cc,from,to
0,0.097826,0.304348,0.597826
1,0.000000,0.571429,0.428571
2,0.252336,0.345794,0.401869
3,0.218069,0.352025,0.429907
4,0.258065,0.161290,0.580645


In [33]:
densities.apply(entropy, axis=1).mean()

0.8574269985217116

# Next?

Possible next steps for this software include refactoring the internals under pandas and implementation of alternative MCMC schemes, possibly including vertex-labeled ones. 